### UNET-seResNet18 segmentation network : Segmentation

This Notebook segment one full (1024x1024) image from T0 full image dataset using a trained model on T0 dataset.

Feel free to change the *model_name* and *path_test_images* to process to another dataset into the cell below.

Path to the trained model and the dataset

In [ ]:
# T0 segmentation model
model_name = "models/T0/jerome_128x128_seresnet18_e20_spe400.h5"

# T0 full image
path_test_images = "data/T0/fullimage/"

In [ ]:
import os
import numpy as np
from PIL import Image
import segmentation_models as sm
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.optimizers import Adam
from keras.models import load_model
import matplotlib.pyplot as plt
from tifffile import imsave

Useful functions

In [ ]:
def load_images_from_directory(folder,resize,filextension):
    '''
    This function read images from a directory and store it into a Numpy array
    
    folder : path to the folder containing the images
    resize : target size of the images stored into the Numpy array
    filextension: image format. For the moment only 'tif' images are accepted
    
    Use : X = load_images_from_directory( foo/bar/, (128,128), 'tif')
    
    '''
    images  = []
    img_rows=resize[0]
    img_cols=resize[1]
    for filename in sorted(os.listdir(folder)):
        curimg = os.path.join(folder, filename)
        if curimg.endswith(filextension):
            img = Image.open(curimg)
            resize = img.resize((img_rows,img_cols), Image.NEAREST)
            images.append(resize)
    imgarray=list();
    for i in range(len(images)):
        tmp = np.array(images[i])
        imgarray.append(tmp)
    imgarray = np.asarray(imgarray).astype('float32')

    if len(imgarray.shape)==3:
        imgarray = np.expand_dims(imgarray,axis=3)

    return imgarray

Parameters

In [ ]:
target_size=(128,128)

BACKBONE = "seresnet18"

bs = 32

Load a trained segmentation model

In [ ]:
K.clear_session()

model = load_model(model_name,custom_objects={
      "dice_loss":sm.losses.dice_loss,
      "iou_score":sm.metrics.iou_score}
      )


Read, process and segment the full size image (1024x1024) by combining the model prediction on (128x128) patches

In [ ]:
A = load_images_from_directory(path_test_images,(1024,1024),'tif')

preprocess_input = sm.get_preprocessing(BACKBONE)


R=np.zeros(A.shape)

nx = A.shape[1]//target_size[0]
ny = A.shape[2]//target_size[1]

tx=target_size[0]
ty=target_size[1]

for i in range(nx):
    for j in range(ny):
        a = A[:,i*tx:i*tx+tx,j*ty:j*ty+ty,:]
        X_test = np.concatenate((a,a,a),axis=3)
        X_test = preprocess_input(X_test)
        p = model.predict(X_test)
        R[:,i*tx:i*tx+tx,j*ty:j*ty+ty,:]=p
        

Export the full segmented image

In [ ]:
imsave("T0_full_segmented_image.tif",(R>0.5)*255)